In [ ]:
%pip install comet_ml #libreria a instalar

In [ ]:
import comet_ml
comet_ml.init(project_name="RNA_Densa_L1L2") #nombre a dar al experimento

Please paste your Comet API key from https://www.comet.com/api/my/settings/
(api key may not show as you type)
Comet API key: ··········


COMET INFO: Valid Comet API Key saved in /root/.comet.config (set COMET_CONFIG to change where it is saved).


In [ ]:
# Imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist #base de datos
from tensorflow.keras.models import Sequential # checar cada uno de los que faltan
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Input
from tensorflow.keras.optimizers import RMSprop, SGD, Adam
from tensorflow.keras import regularizers
from keras.callbacks import ModelCheckpoint # sirve para dar respuesta del proceso de entrenamiento de la red

In [ ]:
experiment = comet_ml.Experiment(  # para activar las Graficas de comet
    auto_histogram_weight_logging=True,
    auto_histogram_gradient_logging=True,
    auto_histogram_activation_logging=True,
    log_code=True, # para subir el codigo a comet para no perder el progreso
)

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/nicglowss/rna-densa-l1l2/cd2fa02a1b4b401f87e65d63237bf43f
COMET INFO:   Uploads:
COMET INFO:     environment details : 1
COMET INFO:     filename            : 1
COMET INFO:     installed packages  : 1
COMET INFO:     notebook            : 2
COMET INFO:     os packages         : 1
COMET INFO:     source_code         : 1
COMET INFO: 
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repos

In [ ]:
# pre-procesamiento, aqui normalizamos la informacion de los pixeles
# Lo principal con la red es tener la misma normalizacion y tamaño de imagen.
dataset=mnist.load_data()# en el dataset se guardan todas las imagenes del mnist con sus etiquetas
(x_train, y_train), (x_test, y_test) = dataset

# se van a guardar en el forato de binario
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# =numero de pixeles normalizando el no. de pixeles
x_train /= 255  # x_trainv = x_trainv/255
x_test /= 255

#clasificacion de clases
num_classes=10
y_trainc = keras.utils.to_categorical(y_train, num_classes) #analiza los datos y los acomoda en las distintas clases
y_testc = keras.utils.to_categorical(y_test, num_classes)

11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
# Parametros
parameters = {
    "batch_size": 10,
    "epochs": 30,
    "optimizer": "SGD",
    "loss": "categorical_crossentropy",
}

experiment.log_parameters(parameters) # diccionario subido a comet

In [ ]:
#Arquitetura de la red
model = Sequential()
# aqui empezamos
model.add(Input(shape=(28,28))) #Flaten no tiene la opcion input_shape por lo tanto se tiene que agregar esta capa
                                #28x28 es el tamaño de la imagen
model.add(Flatten()) #Otra forma de aplanar las imagenes, las forma en una linea
model.add(Dense(784, activation='sigmoid', kernel_regularizer=regularizers.L1L2(0.01,0.00001)))
                                            # esta es la capa densa de no neuronas de forma vertical (en este caso agarramos el total de pixeles)
                                            # su funcion de activacion es la sigmoide (respuesta de la neurona de 0 a 1)
                                            # Estas dos son las principales

model.add(Dense(10, activation='softmax'))  # son las de salida literal porque estan al final

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 784)               615440    
                                                                 
 dense_1 (Dense)             (None, 10)                7850      
                                                                 
Total params: 623290 (2.38 MB)
Trainable params: 623290 (2.38 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# specify the path where you want to save the model
filepath = "mejor-modelo1.1.hdf5"

# initialize the ModelCheckpoint callback
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [ ]:
# Inicia el entrenamiento de la red
model.compile(loss=parameters['loss'],optimizer='SGD',metrics=['accuracy'])
historial=model.fit(x_train, y_trainc,
                    batch_size=parameters['batch_size'],
                    epochs=parameters["epochs"],
                    verbose=1,
                    validation_data=(x_test, y_testc),
                    callbacks=[checkpoint])

COMET INFO: Ignoring automatic log_parameter('verbose') because 'keras:verbose' is in COMET_LOGGING_PARAMETERS_IGNORE


Epoch 1/30
6000/6000 [==============================] - ETA: 0s - loss: 9.1949 - accuracy: 0.1151
Epoch 1: val_loss improved from inf to 2.68911, saving model to mejor-modelo1.1.hdf5
6000/6000 [==============================] - 76s 12ms/step - loss: 9.1949 - accuracy: 0.1151 - val_loss: 2.6891 - val_accuracy: 0.0892
Epoch 2/30
   1/6000 [..............................] - ETA: 1:28 - loss: 2.6561 - accuracy: 0.1000

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


5999/6000 [============================>.] - ETA: 0s - loss: 2.6637 - accuracy: 0.1030
Epoch 2: val_loss improved from 2.68911 to 2.64347, saving model to mejor-modelo1.1.hdf5
6000/6000 [==============================] - 63s 10ms/step - loss: 2.6637 - accuracy: 0.1030 - val_loss: 2.6435 - val_accuracy: 0.1010
Epoch 3/30
5998/6000 [============================>.] - ETA: 0s - loss: 2.6622 - accuracy: 0.1035
Epoch 3: val_loss did not improve from 2.64347
6000/6000 [==============================] - 63s 11ms/step - loss: 2.6622 - accuracy: 0.1035 - val_loss: 2.6603 - val_accuracy: 0.1009
Epoch 4/30
5995/6000 [============================>.] - ETA: 0s - loss: 2.6618 - accuracy: 0.1033
Epoch 4: val_loss did not improve from 2.64347
6000/6000 [==============================] - 69s 12ms/step - loss: 2.6619 - accuracy: 0.1033 - val_loss: 2.6701 - val_accuracy: 0.1028
Epoch 5/30
5996/6000 [============================>.] - ETA: 0s - loss: 2.6624 - accuracy: 0.1023
Epoch 5: val_loss did not impro

In [ ]:
experiment.log_model("MNIST1", "mejor-modelo1.1.hdf5")

{'web': 'https://www.comet.com/api/asset/download?assetId=a9395a73f00a459bb192ee3fca5c28d5&experimentKey=57de4a0dc22e417d983239a8ee513f0a',
 'api': 'https://www.comet.com/api/rest/v2/experiment/asset/get-asset?assetId=a9395a73f00a459bb192ee3fca5c28d5&experimentKey=57de4a0dc22e417d983239a8ee513f0a',
 'assetId': 'a9395a73f00a459bb192ee3fca5c28d5'}

In [ ]:
experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/nicglowss/rna-densa-l1l2/57de4a0dc22e417d983239a8ee513f0a
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [30]                  : (0.10209999978542328, 0.11509999632835388)
COMET INFO:     batch_accuracy [18000]         : (0.0, 0.4000000059604645)
COMET INFO:     batch_loss [18000]             : (2.4404618740081787, 192.947021484375)
COMET INFO:     epoch_duration [30]            : (52.65089023999997, 129.65062087299998)
COMET INFO:     loss [30]                      : (2.6577565670013428, 9.19492244720459)
COMET INFO:     val_accuracy [30]              : (0.08919999748468399, 0.11349999904632568)
COMET INFO: 